In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random
import os
import pickle

from scipy.sparse import csr_matrix, save_npz
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from source import *
from utils import draw_barplor
from recomendations import make_recomendations_with_cf, make_recomendations_with_genre, make_recomendations_with_sypnopsis

## Предобработка данных

### работа с признаками

In [ ]:
anime_ratings = pd.read_csv(ANIME_DIR + "animelist.csv", nrows=10000000)
anime_data = pd.read_csv(ANIME_DIR + "anime.csv")

Рассиморим, какая информация находится в файле $anime.csv$

In [ ]:
anime_data.info()

Сразу переименуем колонку $MAL\_ID$ в $anime\_id$

In [ ]:
anime_data.rename(columns={'MAL_ID':"anime_id"},inplace=True)
anime_data.columns

Избавимся от лишних полей

In [ ]:
to_keep = ['anime_id', 'Name', 'Score', 'Genres', 'Members']
anime_data = anime_data[to_keep]
anime_data

Сейчас у нас все жанры описаны в одном поле через запятую, что не очень удобно, поэтому мы определим все жанры и добавим их как поля для каждой записи

In [ ]:
genres_column = anime_data["Genres"].map(lambda x: x.split(", "))
genres = list(set(sum(genres_column, [])))

anime_data[genres] = 0
for i in range(0, len(genres_column)):
    anime_data.loc[i, genres_column[i]] = 1

anime_data = anime_data.drop(columns="Genres")

Заменим Unknown в поле Score на 0

In [ ]:
dict = {'Unknown' : 0}
anime_data['Score'] = anime_data['Score'].astype(str).apply(lambda x : dict[x] if x == 'Unknown' else x).astype(float)

anime_data.info()

Рассмотрим теперь информацию файла $animelist.csv$

In [ ]:
anime_ratings.info()

Избавимся от информации о кол-ве просмотренных эпизодов и статусе просмотра

In [ ]:
anime_ratings = anime_ratings[['user_id', 'anime_id', 'rating']]
anime_ratings.info()

Проверим, есть ли оценки для всех аниме, представленных в датасете

In [ ]:
anime_ratings.anime_id.nunique()

Это действительно так

Объеденим информацию из двух файлов

In [ ]:
anime_complete = pd.merge(anime_data, anime_ratings, on='anime_id')
anime_complete.info()

Переименуеем Score в total_score, а rating в user_score

In [ ]:
anime_complete = anime_complete.rename(columns={'Score' : 'total_score', 'rating': 'user_score'})

anime_complete.isna().sum()

сохраним полученый df в csv

In [ ]:
anime_complete.to_csv(ANIME_DIR + 'complete.csv')

### Подготовка данных для рекомендаций

In [ ]:
anime_feature = pd.read_csv(ANIME_DIR + 'complete.csv')
anime_feature = anime_feature.drop(columns='Unnamed: 0')
anime_feature.head()

Ради интереса посмотрим на 10 самых популярных аниме по кол-ву оценок и по кол-ву фанатов

In [ ]:
top10_by_score = anime_feature['Name'].value_counts().nlargest(10)
top10_by_members = anime_feature.sort_values(by='Members', ascending=False).drop_duplicates(subset='Name').head(10)

In [ ]:
draw_barplor(top10_by_score.index, top10_by_score.values, 
             "топ 10 по суммарному рейтингу", "название аниме", "суммарный рейтинг")

И тут в поезде анимешников завязалась драка...

In [ ]:
draw_barplor(top10_by_members['Name'], top10_by_members['Members'],
             "топ 10 по числу фанатов", "название аниме", "фанаты")

...в которую ворвались адепты Всемогущего, Сайтамы и 1000 - 7 ...

Проверим сколько в среднем поставил оценок каждый пользователь

In [ ]:
count_of_users = anime_feature['user_id'].value_counts()
count_of_users.describe()

Половина пользователей, оцекни которых мы собираемся использовать поставили
оценки меньше чем $67$ анииме, однако средним значением для выборки является около
$101$-ой оценки. Если рассматривать, что большая часть оценок поставлена по
просмотру аниме и при этом не для каждого просмотренного аниме пользователь
поставил оценку, то для дальнейшей работы
стоит выбрать пользователей, которые поставили оцеку $75$ и более аниме. 

Данное значение было взято из следующего:
- Мной просмотрено около 300 аниме, но при этом оценка выставлена лишь половине.
- Большая часть выходящих аниме преставляют собой 12-ти серийные сериалы,
  средняя продолжительность которых составляет $12 * 24 / 60 = 4.8$ часа.
- Следовательно человек, просмотревший $75$ аниме, потратил на это $360$ часов...

$360$ часов можно интерпретировать примерно как год работы кинокритика, т.к
кроме самого просмотра, человек ещё тратит какое-то время на осмысление сюжета,
понимание мотивов героев. Если он конечно смотрит их не залпом по несколько
аниме в день из-за кошкодевочек и им подобным)

In [ ]:
print(f'{type(count_of_users)}\n{count_of_users}')

In [ ]:
anime_feature = anime_feature[anime_feature['user_id'].isin(count_of_users[count_of_users >= 75].index)]
anime_feature.user_id.nunique()

создадим теперь таблицу, в которой строками будут названия аниме, а столбцами id пользователей. Значениями будут оценки

In [ ]:
anime_pivot = anime_feature.pivot_table(index='Name', columns='user_id',
                                        values='user_score').fillna(0)
anime_pivot.head()

также сохраним его для дальнейшей работы, предварительно преобразовав в csr_matrix для удобства разворачивания в дальнейшем

In [ ]:
cf_matrix = csr_matrix(anime_pivot.values)
save_npz(ANIME_DIR + 'cf_matrix.npz', cf_matrix)

## Коллаборативная рекомендация

In [ ]:
# cf_matrix = load_npz(ANIME_DIR + 'cf_matrix.npz')
anime_pivot = pd.read_csv(ANIME_DIR + 'anime_cf.csv', index_col='Name')
anime_pivot.head()

In [ ]:
anime_pivot.info()

Слишком много места требуется для данных. Выберем 4000 случайных пользователей и их оценки.

In [ ]:
copy_pivot = anime_pivot[random.sample(anime_pivot.columns.to_list(), 4000)]
copy_pivot.info()

In [ ]:
cf_matrix = csr_matrix(copy_pivot.values)

In [ ]:
knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(cf_matrix)

In [ ]:
anime_titles = copy_pivot.index
anime_title = np.random.choice(anime_titles)
query_index = copy_pivot.index.get_loc(anime_title)
print(f"Randomly selected anime title: {anime_title} \n")

In [ ]:
distances, index = knn_model.kneighbors(copy_pivot.iloc[query_index, :]
                                        .values.reshape(1, -1)
                                        , n_neighbors=10)

print(f"Recommendations for {anime_pivot.index[query_index]}:\n")
for i, ind in enumerate(index.flatten()):
    print(f'''{i + 1}: {copy_pivot.index[ind]}, with distance {distances.flatten()[i]:{4}.{3}}''')

In [ ]:
result = make_recomendations_with_cf(copy_pivot, count_recomendations=20, model_path= 'models/cf_model.sav')

if result != None:
    print(f' {20} title recomendations for {result[0]}')
    for i, value in enumerate(result[1]):
        print(f'{i + 1}: {value[0]}, with distance {value[1]:{4}.{3}}')

In [ ]:
result = make_recomendations_with_cf(copy_pivot, count_recomendations=20, model_path= 'cf_model.sav', name='aboba')

if result != None:
    print(f' {20} title recomendations for {result[0]}')
    for i, value in enumerate(result[1]):
        print(f'{i + 1}: {value[0]}, with distance {value[1]:{4}.{3}}')
else:
    print('Anime not found')

## Рекомендации на основе жанра, описания

У метода коллаборативной фильтрации есть несколько минусов:
1. Объем данных
2. Существуют такие аниме, для которых есть слишком мало оценок, поэтому
   рекомендации выстраиваются на основе предпочтений тех людей, которые
   просмотрели это аниме и это не дает нам реальной пользы, т.к рекомендации
   начинают работать на основе вкуса 1-2 людей

In [2]:
synopsis_data = pd.read_csv(ANIME_DIR + 'anime_with_synopsis.csv')
synopsis_data.rename(columns={'MAL_ID':"anime_id"},inplace=True)
synopsis_data.head()

,anime_id,Name,Score,Genres,sypnopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...


In [ ]:
synopsis_data.info()

634 KB звучит гораздо лучше, чем 500 MB при условии, что это вес используемого среза оценок по пользователям

In [ ]:
synopsis_data['Genres'] = synopsis_data['Genres'].fillna('')
genres = synopsis_data['Genres'].str.split(', ').astype(str)

In [ ]:
tfidfv = TfidfVectorizer()
tfidf_genres = tfidfv.fit_transform(genres)

In [ ]:
cos_sim = cosine_similarity(tfidf_genres, tfidf_genres)
type(cos_sim)

In [ ]:
anime_indexes = pd.Series(synopsis_data.index, index=synopsis_data['Name'])
anime_indexes

In [ ]:
title = np.random.choice(anime_indexes.index)
title

In [ ]:
cos_scores = sorted(list(enumerate(cos_sim[anime_indexes[title]]))
                    , key= lambda x: x[1]
                    , reverse=True)
recomendations = [i[0] for i in cos_scores[0:10]]
similarity = [i[1] for i in cos_scores[0:10]]

print(f'Recomendations for {title}:') 

for i, value in enumerate(recomendations):
        print(f'''{i + 1}: {synopsis_data["Name"]
                            .iloc[value]}, with similarity {similarity[i]:{4}.{3}}''')
    

In [ ]:
result = make_recomendations_with_genre(synopsis_data, name=title)

if result != None:
    print(f' Recomendations for {result[0]}')
    
    for i, value in enumerate(result[1]):
        print(f'{i + 1}: {value[0]}, with similarity in genre {value[1]:{4}.{3}}')
else:
    print('Anime not found')

In [3]:
result = make_recomendations_with_genre(synopsis_data
                                        , name='Shingeki no Kyojin'
                                        , count_recomendations= 20)

if result != None:
    print(f' Recomendations for {result[0]}')
    for i, value in enumerate(result[1]):
        print(f'{i + 1}: {value[0]}, with similarity in genre {value[1]:{4}.{3}}')
else:
    print('Anime not found')

 Recomendations for Shingeki no Kyojin
1: Shingeki no Kyojin Season 2, with similarity in genre  1.0
2: Shingeki no Kyojin Season 3, with similarity in genre  1.0
3: Shingeki no Kyojin Season 3 Part 2, with similarity in genre  1.0
4: Shingeki no Kyojin: The Final Season, with similarity in genre  1.0
5: Shingeki no Kyojin: Chronicle, with similarity in genre  1.0
6: Shingeki no Kyojin OVA, with similarity in genre 0.806
7: Shingeki no Kyojin: Ano Hi Kara, with similarity in genre 0.806
8: Shingeki no Kyojin Movie 1: Guren no Yumiya, with similarity in genre 0.806
9: Shingeki no Kyojin Movie 2: Jiyuu no Tsubasa, with similarity in genre 0.806
10: Shingeki no Kyojin Season 2 Movie: Kakusei no Houkou, with similarity in genre 0.806
11: GetBackers, with similarity in genre 0.787
12: Speed Grapher, with similarity in genre 0.76
13: Skull Man, with similarity in genre 0.76
14: Saint Seiya: Meiou Hades Elysion-hen, with similarity in genre 0.755
15: Dragon Ball Z: The Real 4-D, with similari

... А что если проверить что-то, у чего нет 10+ частей?

In [4]:
result = make_recomendations_with_genre(synopsis_data
                                        , name='Death Note'
                                        , count_recomendations= 20)

if result != None:
    print(f' Recomendations for {result[0]}')
    for i, value in enumerate(result[1]):
        print(f'{i + 1}: {value[0]}, with similarity in genre {value[1]:{4}.{3}}')
else:
    print('Anime not found')

 Recomendations for Death Note
1: Death Note: Rewrite, with similarity in genre 0.961
2: B: The Beginning, with similarity in genre 0.937
3: B: The Beginning Succession, with similarity in genre 0.937
4: Yakusoku no Neverland 2nd Season: Michishirube, with similarity in genre 0.883
5: Mousou Dairinin, with similarity in genre 0.861
6: Mirai Nikki, with similarity in genre 0.857
7: Higurashi no Naku Koro ni Kai, with similarity in genre 0.839
8: Higurashi no Naku Koro ni Rei, with similarity in genre 0.82
9: Zankyou no Terror, with similarity in genre 0.779
10: Mi Yu Xing Zhe, with similarity in genre 0.779
11: Babylon, with similarity in genre 0.779
12: Imawa no Kuni no Alice (OVA), with similarity in genre 0.772
13: Mouryou no Hako, with similarity in genre 0.771
14: Monster, with similarity in genre 0.757
15: Yakusoku no Neverland 2nd Season, with similarity in genre 0.756
16: Black Jack, with similarity in genre 0.74
17: AD Police, with similarity in genre 0.728
18: Higurashi no Nak

Однако все равно сходство на основе жанра не может нам гарантировать то, что
рекомендоваться будет то, что действительно соответствует желаниям пользователя.

In [5]:
result = make_recomendations_with_genre(synopsis_data
                                        , name='Seishun Buta Yarou wa Bunny Girl Senpai no Yume wo Minai'
                                        , count_recomendations= 20)

if result != None:
    print(f' Recomendations for {result[0]}')
    for i, value in enumerate(result[1]):
        print(f'{i + 1}: {value[0]}, with similarity in genre {value[1]:{4}.{3}}')
else:
    print('Anime not found')

 Recomendations for Seishun Buta Yarou wa Bunny Girl Senpai no Yume wo Minai
1: Nakitai Watashi wa Neko wo Kaburu, with similarity in genre  1.0
2: Wind: A Breath of Heart (TV), with similarity in genre 0.955
3: Wind: A Breath of Heart OVA, with similarity in genre 0.955
4: Aura: Maryuuin Kouga Saigo no Tatakai, with similarity in genre 0.955
5: Kimi no Na wa., with similarity in genre 0.955
6: Seishun Buta Yarou wa Yumemiru Shoujo no Yume wo Minai, with similarity in genre 0.955
7: Rokujouma no Shinryakusha!?, with similarity in genre 0.906
8: Otome wa Boku ni Koishiteru, with similarity in genre 0.858
9: Keitai Shoujo, with similarity in genre 0.858
10: Ajimu: Kaigan Monogatari, with similarity in genre 0.858
11: Kotoura-san, with similarity in genre 0.858
12: Suki ni Naru Sono Shunkan wo.: Kokuhaku Jikkou Iinkai, with similarity in genre 0.858
13: Itsudatte Bokura no Koi wa 10 cm Datta., with similarity in genre 0.858
14: Rewrite, with similarity in genre 0.841
15: Shakugan no Shana

Получилось не совсем то, что планировалось, но всё же. Для здоровых людей:
- Аниме(конкретно для 12-ти серийника) для которого требовалось найти похожих
  --- представляет собой школьный ромком с небольшим количеством
  сверхестественного.
- Первой рекомендацией получили аниме, название которого само говорит за себя:
  <<Сквозь слезы я притворяюсь кошкой>>.

Проверим теперь рекомендации, основываясь на описании к тайтлу.

In [6]:
result = make_recomendations_with_sypnopsis(synopsis_data
                                        , name='Seishun Buta Yarou wa Bunny Girl Senpai no Yume wo Minai'
                                        , count_recomendations= 20)

if result != None:
    print(f' Recomendations for {result[0]}')
    for i, value in enumerate(result[1]):
        print(f'{i + 1}: {value[0]}, with similarity {value[1]:{4}.{3}}')
else:
    print('Anime not found')

 Recomendations for Seishun Buta Yarou wa Bunny Girl Senpai no Yume wo Minai
1: Seishun Buta Yarou wa Yumemiru Shoujo no Yume wo Minai, with similarity 0.28
2: Märchen Mädchen, with similarity 0.232
3: Anime Himitsu no Hanazono, with similarity 0.223
4: Kamichama Karin, with similarity 0.221
5: Mahou no Star Magical Emi, with similarity 0.216
6: Kumo Desu ga, Nani ka?, with similarity 0.216
7: Gotou ni Naritai., with similarity 0.214
8: Kirara, with similarity 0.207
9: Mirai Nikki: Redial, with similarity 0.207
10: Amnesia, with similarity 0.206
11: Mahou Shoujo Site, with similarity 0.206
12: Joshikousei no Mudazukai, with similarity 0.206
13: Kareshi Kanojo no Jijou, with similarity 0.206
14: Hirune Hime: Shiranai Watashi no Monogatari, with similarity 0.205
15: Wotaku ni Koi wa Muzukashii, with similarity 0.205
16: Code-E, with similarity 0.201
17: Gakkougurashi!, with similarity  0.2
18: Hiiro no Kakera, with similarity 0.199
19: Cyclops Shoujo Saipuu, with similarity 0.199
20: Lov

Вроде, по моему личному опыту, это выглядит немного интереснее и более близко к тематике первоначального аниме.

In [7]:
result = make_recomendations_with_sypnopsis(synopsis_data
                                        , name='Death Note'
                                        , count_recomendations= 20)

if result != None:
    print(f' Recomendations for {result[0]}')
    for i, value in enumerate(result[1]):
        print(f'{i + 1}: {value[0]}, with similarity {value[1]:{4}.{3}}')
else:
    print('Anime not found')

 Recomendations for Death Note
1: Death Note: Rewrite, with similarity 0.314
2: Soul Eater, with similarity 0.251
3: Shinigami no Ballad., with similarity 0.245
4: YAT Anshin! Uchuu Ryokou 2, with similarity 0.222
5: Dia Horizon (Kabu), with similarity 0.207
6: Persona 3 the Movie 4: Winter of Rebirth, with similarity 0.206
7: Yami no Matsuei, with similarity 0.202
8: Zombie-Loan, with similarity 0.198
9: Bleach: Memories in the Rain, with similarity 0.197
10: Shiki, with similarity 0.196
11: Sword Art Online II, with similarity 0.196
12: Neppuu Kairiku Bushi Road, with similarity 0.196
13: Shironeko Project: Zero Chronicle, with similarity 0.192
14: Gantz:O, with similarity 0.189
15: Choujuu Densetsu Gestalt, with similarity 0.188
16: Koutetsujou no Kabaneri, with similarity 0.188
17: Renkin San-kyuu Magical? Pokaan, with similarity 0.187
18: Utsunomiko: Heaven Chapter, with similarity 0.187
19: Nissan Note x The World of Golden Eggs, with similarity 0.185
20: Bungou Stray Dogs 3rd Se

Рассмотрим на более популярном аниме...

In [8]:
result = make_recomendations_with_genre(synopsis_data
                                        , name='Sword Art Online'
                                        , count_recomendations= 20)

if result != None:
    print(f' Recomendations for {result[0]}')
    for i, value in enumerate(result[1]):
        print(f'{i + 1}: {value[0]}, with similarity {value[1]:{4}.{3}}')
else:
    print('Anime not found')

 Recomendations for Sword Art Online
1: Sword Art Online: Extra Edition, with similarity  1.0
2: Sword Art Online II, with similarity  1.0
3: Sword Art Online Movie: Ordinal Scale, with similarity  1.0
4: Chou Yuu Sekai: Being the Reality, with similarity  1.0
5: Sword Art Online: Alicization, with similarity  1.0
6: Sword Art Online: Alicization - War of Underworld, with similarity  1.0
7: Sword Art Online: Alicization - War of Underworld Reflection, with similarity  1.0
8: Sword Art Online: Alicization - War of Underworld 2nd Season, with similarity  1.0
9: Sword Art Online: Alicization - War of Underworld Recap, with similarity  1.0
10: Sword Art Online: Progressive Movie - Hoshi Naki Yoru no Aria, with similarity  1.0
11: Ys IV: The Dawn of Ys, with similarity  0.9
12: Sword Art Online II: Debriefing, with similarity  0.9
13: Valhait Rising: Kandou e., with similarity  0.9
14: Slime Boukenki: Umi da, Yeah!, with similarity 0.86
15: Battle Spirits: Sword Eyes, with similarity 0.834


Мы получили 0 аниме, похожих хоть как-то на SAO(не считая его частей)...

In [9]:
result = make_recomendations_with_sypnopsis(synopsis_data
                                        , name='Sword Art Online'
                                        , count_recomendations= 20)

if result != None:
    print(f' Recomendations for {result[0]}')
    for i, value in enumerate(result[1]):
        print(f'{i + 1}: {value[0]}, with similarity {value[1]:{4}.{3}}')
else:
    print('Anime not found')

 Recomendations for Sword Art Online
1: Sword Art Online: Progressive Movie - Hoshi Naki Yoru no Aria, with similarity 0.459
2: Sword Art Online Movie: Ordinal Scale, with similarity 0.395
3: Sword Art Online II, with similarity 0.354
4: Sword Art Online: Alicization - War of Underworld 2nd Season, with similarity  0.3
5: Sword Art Online: Extra Edition, with similarity 0.296
6: Kyuukyoku Shinka shita Full Dive RPG ga Genjitsu yori mo Kusoge Dattara, with similarity 0.26
7: WIXOSS Diva(A)Live, with similarity 0.254
8: Btooom!, with similarity 0.247
9: Genei Toushi Bastof Lemon, with similarity 0.24
10: Omoikkiri Kagaku Adventure Sou Nanda!, with similarity 0.239
11: Ryuu ga Gotoku Online x Taka no Tsume, with similarity 0.238
12: Log Horizon, with similarity 0.237
13: Sword Art Online: Alicization, with similarity 0.234
14: Sword Art Online: Alicization - War of Underworld, with similarity 0.227
15: Kyokugen Dasshutsu Adv: Zennin Shibou Desu Prologue, with similarity 0.227
16: Sword Ar

Это выглядит гораздо лучше:
- Kyuukyoku Shinka shita Full Dive RPG ga Genjitsu yori mo Kusoge Dattara ---
  очень похоже идейно(за исключением попаданства) на SAO и связано с игрой в vr.
- Btooom! --- попаданец в игру...
- Log Horizon --- попаданцы в игру + гг обоих аниме имеют какую-то общую
  особенность + чуточку романтикии...